In [4]:
import os
import numpy as np
import re
import pandas as pd
import scipy as sp
import random
import math

## Read and parse TSP data  

In [5]:
data = {}
for file in os.listdir('data'):
    with open(f'data/{file}') as f:
        name = f.readline().strip().split(' : ')[1]
        dim = f.readline().strip().split(' : ')[1]

        raw = f.readlines()
        raw = [list(map(float, re.split(r"\s+", line.strip()))) for line in raw]

        data[name] = raw

In [6]:
keys = data.keys()

## a) Create distance matrix

In [7]:
matrices = dict.fromkeys(keys, [])

for key in keys:
    dist = sp.spatial.distance_matrix(data[key], data[key])
    matrices[key] = dist

## Define Simple and Stochastic Hill Climbing algorithms

In [8]:
def generate_initial_tour(num_cities):
    tour = list(range(num_cities))
    random.shuffle(tour)
    return tour

def calculate_tour_length(tour, distances):
    length = 0
    for i in range(len(tour) - 1):
        length += distances[tour[i]][tour[i + 1]]
    length += distances[tour[-1]][tour[0]]
    return length

def get_neighbor(tour):
    i, j = random.sample(range(len(tour)), 2)
    tour[i], tour[j] = tour[j], tour[i]
    return tour

def simple_hill_climb(distances):
    num_cities = len(distances)
    current_tour = generate_initial_tour(num_cities)
    current_length = calculate_tour_length(current_tour, distances)

    for _ in range(10000):
        neighbor = get_neighbor(current_tour.copy())
        neighbor_length = calculate_tour_length(neighbor, distances)

        if neighbor_length < current_length:
            current_tour = neighbor
            current_length = neighbor_length

    return current_tour, current_length

def stochastic_hill_climb(distances):
    num_cities = len(distances)
    current_tour = generate_initial_tour(num_cities)
    current_length = calculate_tour_length(current_tour, distances)
    temperature = 1.0

    for _ in range(10000):
        neighbor = get_neighbor(current_tour.copy())
        neighbor_length = calculate_tour_length(neighbor, distances)

        delta_length = neighbor_length - current_length
        if delta_length < 0 or random.random() < pow(math.e, -(delta_length / temperature)):
            current_tour = neighbor
            current_length = neighbor_length

        temperature *= 0.95

    return current_tour, current_length

## b) Run the algorithms on generated matrices

In [9]:
final = {"Dataset": [], "Nodes explored": [] ,"Simple Hill Climb cost": [], "Stochastic Hill Climb cost": []}
for key, val in data.items():
    print(key)
    print()
    print("Simple Hill Climb")
    tour, shc_length = simple_hill_climb(matrices['rd100'])
    print(f"Tour: {tour}\nLength: {shc_length}")
    print()
    print("Stochastic Hill Climb")
    tour, sthc_length = stochastic_hill_climb(matrices['rd100'])
    print(f"Tour: {tour}\nLength: {sthc_length}")
    print("\n\n")

    final["Dataset"].append(key)
    final["Nodes explored"].append(len(tour))
    final["Simple Hill Climb cost"].append(shc_length)
    final["Stochastic Hill Climb cost"].append(sthc_length)

a280

Simple Hill Climb
Tour: [68, 14, 20, 2, 5, 93, 30, 44, 43, 33, 6, 41, 23, 24, 42, 39, 83, 87, 97, 46, 56, 51, 95, 38, 54, 35, 48, 74, 3, 31, 77, 19, 32, 63, 60, 4, 80, 21, 50, 13, 11, 73, 72, 37, 25, 8, 59, 0, 17, 61, 86, 84, 81, 96, 85, 62, 66, 12, 9, 26, 55, 49, 45, 36, 18, 75, 65, 22, 34, 28, 99, 7, 70, 67, 82, 79, 64, 47, 29, 40, 10, 91, 16, 71, 69, 53, 27, 76, 92, 58, 94, 78, 98, 15, 1, 88, 57, 52, 90, 89]
Length: 17504.848196283514

Stochastic Hill Climb
Tour: [73, 37, 72, 53, 69, 71, 16, 19, 78, 5, 65, 83, 87, 94, 92, 36, 27, 76, 58, 75, 44, 51, 10, 33, 6, 41, 4, 80, 89, 17, 61, 86, 13, 11, 3, 2, 32, 9, 63, 90, 82, 67, 70, 59, 20, 74, 81, 84, 85, 96, 62, 48, 97, 15, 99, 23, 24, 42, 39, 43, 34, 28, 38, 95, 54, 60, 12, 14, 0, 7, 64, 21, 40, 35, 56, 46, 49, 45, 55, 26, 79, 29, 47, 50, 68, 66, 98, 52, 93, 30, 22, 1, 88, 57, 18, 91, 25, 8, 31, 77]
Length: 16226.53116448578



ch150

Simple Hill Climb
Tour: [97, 44, 43, 34, 28, 95, 99, 46, 98, 11, 13, 74, 20, 81, 84, 14, 67, 29,

## c) Tabulate resultant tour length and cost data

In [10]:
df = pd.DataFrame(final)

In [11]:
df

,Dataset,Nodes explored,Simple Hill Climb cost,Stochastic Hill Climb cost
0,a280,100,17504.848196,16226.531164
1,ch150,100,16800.678144,15406.016654
2,d198,100,18849.448741,17218.429559
3,eil101,100,15428.667968,17026.693737
4,rd100,100,18128.531928,16205.218311
